Каспаров Иоанн 6132

Лабораторная работа 1

Перемножение матриц

In [ ]:
!/usr/local/cuda/bin/nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!cuda-install-samples-11.2.sh ~ & cd /root/NVIDIA_CUDA-11.2_Samples/

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-t6fwg3fv
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-t6fwg3fv
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0d2ab99cccbbc682722e708515fe9c4cfc50185a
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4716 sha256=ee97aafe11ea6195e9b9e7e4eb250bf8cb272e8938411b68fa2e9a62d3ac68ef
  Stored in directory: /tmp/pip-ephem-wheel-cache-htxqz5gx/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out
/bin/bash: line 1: cd: /root/NVI

In [ ]:
%%cuda --name my_curand.cu

#include <cstdlib>
#include <curand.h>

#include <cuda_runtime.h>
#include <cublas_v2.h>
#include <cuComplex.h>

#include <malloc.h>
#include <stdio.h>
#include <time.h>
#include <stdlib.h>
#include <iostream>

#define IDX2C(i,j,ld) (((j)*(ld))+(i))

using namespace std;

void gpu_blas_mmul(const double *A, const double *B, double *C, const int m, const int k, const int n) {
	int lda = m, ldb = k, ldc = m;
	const double alf = 1;
	const double bet = 0;
	const double *alpha = &alf;
	const double *beta = &bet;

	cublasHandle_t handle; //тип указателя на непрозрачную структуру, содержащую контекст библиотеки cuBLAS
	cublasCreate(&handle); //инициализация контекста библиотеки cuBLAS

	cublasDgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, m, n, k, alpha, A, lda, B, ldb, beta, C, ldc);

	cublasDestroy(handle);
}

void print_matrix(double* matrix, int rows, int cols) {
	for(int i = 0; i < 3; ++i){
		for(int j = 0; j < 3; ++j){
			printf("%f ", matrix[IDX2C(i, j, 3)]);
		}
		printf("\n");
	}
	printf("\n");
}

void sequential(const double* A, const double* B, double*C, const int m, const int k, const int n) {
    for (int i = 0; i < m; ++i) {
        for (int j = 0; j < n; ++j) {
            C[IDX2C(i, j, n)] = 0.0;
            for (int r = 0; r < n; ++r) {
                C[IDX2C(i, j, n)] += A[IDX2C(i, r, k)] * B[IDX2C(r, j, n)];
            }
        }
    }
}

int main() {
	int arr[]={100, 200, 400, 800, 1600, 2000};
	for(int n : arr){
			int nr_rows_A, nr_cols_A, nr_rows_B, nr_cols_B, nr_rows_C, nr_cols_C;
			nr_rows_A = nr_cols_A = nr_rows_B = nr_cols_B = nr_rows_C = nr_cols_C = n;

			double *h_A = (double *)malloc(nr_rows_A * nr_cols_A * sizeof(double));
			double *h_B = (double *)malloc(nr_rows_B * nr_cols_B * sizeof(double));
			double *h_C = (double *)malloc(nr_rows_C * nr_cols_C * sizeof(double));

			int start = 10;
			int end = 30;

			for (int i = 0; i < nr_rows_A * nr_rows_A; i++) {
				h_A[i] = rand() % (end - start + 1) + start;
				h_B[i] = rand() % (end - start + 1) + start;
			}

			double *d_A, *d_B, *d_C;
			cudaMalloc(&d_A, nr_rows_A * nr_cols_A * sizeof(double));
			cudaMalloc(&d_B, nr_rows_B * nr_cols_B * sizeof(double));
			cudaMalloc(&d_C, nr_rows_C * nr_cols_C * sizeof(double));

			cudaMemcpy(d_A, h_A, nr_rows_A * nr_cols_A * sizeof(double), cudaMemcpyHostToDevice);
			cudaMemcpy(d_B, h_B, nr_rows_B * nr_cols_B * sizeof(double), cudaMemcpyHostToDevice);


			srand(time(0));
			double begin1 = clock();
			gpu_blas_mmul(d_A, d_B, d_C, nr_rows_A, nr_cols_A, nr_cols_B);
			double end1 = (clock() - begin1) / CLOCKS_PER_SEC;

			cudaMemcpy(h_C, d_C, nr_rows_C * nr_cols_C * sizeof(double), cudaMemcpyDeviceToHost);
			double *cpu_C = (double *)malloc(nr_rows_C * nr_cols_C * sizeof(double));

			double begin2 = clock();
			sequential(h_A, h_B, cpu_C, nr_rows_A, nr_cols_A, nr_cols_B);
			double end2 = (clock() - begin2) / CLOCKS_PER_SEC;

			printf("-----------------------------\n");
			printf("n = %d\n", n);
			printf("A matrix:\n");
			print_matrix(h_A, nr_rows_A, nr_cols_A);

			printf("B matrix:\n");
			print_matrix(h_B, nr_rows_B, nr_cols_B);

			printf("C matrix on GPU:\n");
			print_matrix(h_C, nr_rows_C, nr_cols_C);
			printf("Time on GPU %f\n\n", end1);

			printf("C matrix on CPU:\n");
			print_matrix(cpu_C, nr_rows_C, nr_cols_C);
			printf("Time on CPU %f\n ", end2);


			cudaFree(d_A);
			cudaFree(d_B);
			cudaFree(d_C);

			free(h_A);
			free(h_B);
			free(h_C);
			free(cpu_C);
	}
	return 0;
}

'File written in /content/src/my_curand.cu'

In [ ]:
!nvcc -o /content/src/my_curand /content/src/my_curand.cu -lcurand -lcublas

In [ ]:

!/content/src/my_curand

-----------------------------
n = 100
A matrix:
11.000000 20.000000 30.000000 
19.000000 25.000000 30.000000 
18.000000 12.000000 13.000000 

B matrix:
14.000000 19.000000 14.000000 
29.000000 20.000000 17.000000 
20.000000 29.000000 25.000000 

C matrix on GPU:
39157.000000 39547.000000 41418.000000 
38060.000000 38979.000000 39810.000000 
41378.000000 39354.000000 41035.000000 

Time on GPU 0.019684

C matrix on CPU:
39157.000000 39547.000000 41418.000000 
38060.000000 38979.000000 39810.000000 
41378.000000 39354.000000 41035.000000 

Time on CPU 0.004420
 -----------------------------
n = 200
A matrix:
14.000000 12.000000 23.000000 
18.000000 29.000000 14.000000 
10.000000 28.000000 16.000000 

B matrix:
19.000000 10.000000 28.000000 
22.000000 16.000000 28.000000 
23.000000 25.000000 29.000000 

C matrix on GPU:
83398.000000 82330.000000 86161.000000 
83613.000000 83095.000000 82409.000000 
83638.000000 86047.000000 82630.000000 

Time on GPU 0.006754

C matrix on CPU:
83398.00000